In [1]:
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
import random
%matplotlib inline

In [2]:
from cook_data import *

In [3]:
val_prc = 10
create_empty = False

In [7]:
source_dir = '/path/to/images'
ds_name = 'my_dataset'
prep_dirs(source_dir, ds_name)

In [8]:
#fill train data
train_img_path = os.path.join(source_dir, ds_name, 'images', 'train')
train_lab_path = os.path.join(source_dir, ds_name, 'labels', 'train')

for path in [d for d in sorted(os.listdir(source_dir)) if d.endswith('_val.json')]:
    path_json = os.path.join(source_dir, path)
    info = viaLabels(path_json)
    path_images = os.path.join(source_dir, os.path.dirname(path), info[0])
    out = via2yola(path_json, path_images, info[1])
    
    for txt in out:
        if create_empty or len(out[txt]) > 0:   
            img_file = os.path.join(path_images, txt[:-4] + '.jpg')
            if os.path.exists(img_file):
                shutil.copy(img_file, os.path.join(train_img_path, txt[:-4] + '.jpg'))
                with open(os.path.join(train_lab_path, txt), 'w') as f:
                    f.write('\n'.join(out[txt]))

convert via annotate for 04_pack: 100%|██████████| 650/650 [00:00<00:00, 676.18it/s]


In [15]:
ds_path = os.path.join(source_dir, ds_name)
with open(ds_path + '.yaml', 'w') as f:
    f.write('train: {}\n'.format(ds_path + os.sep))
    f.write('val: {}\n\n'.format(ds_path + os.sep))
    f.write('nc: {}\n\n'.format(len(info[1].keys())))
    f.write('names: {}\n'.format(list(info[1].keys())))

In [10]:
info[1]

{'0_side': '0',
 '100_bills': '1',
 '200_bills': '2',
 '500_bills': '3',
 '1000_bills': '4',
 '2000_bills': '5',
 '5000_bills': '6',
 '0_box_closed': '7',
 '0_box': '8',
 '100_box_brown': '9',
 '200_box_green': '10',
 '500_box_violet': '11',
 '1000_box_turquoise': '12',
 '2000_box_blue': '13',
 '5000_box_orange': '14'}

In [14]:
#fill val data
val_img_path = os.path.join(source_dir, ds_name, 'images', 'val')
val_lab_path = os.path.join(source_dir, ds_name, 'labels', 'val')

if input('print "1" if need create val with {}% of train_set: '.format(val_prc)) == '1':
    train_set = os.listdir(train_img_path)
    random.shuffle(train_set)

    val_cnt = len(train_set) // val_prc
    val_set = train_set[:val_cnt]

    for vs in val_set:
        shutil.move(os.path.join(train_img_path, vs), os.path.join(val_img_path, vs))
        shutil.move(os.path.join(train_lab_path, vs[:-4] + '.txt'), os.path.join(val_lab_path, vs[:-4] + '.txt'))

print "1" if need create val with 10% of train_set: 1
